In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 101.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.9 MB/s eta 0:00:00


In [2]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.0 MB/s eta 0:00:00


In [3]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer, BitsAndBytesConfig, pipeline

In [4]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    # bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [10]:
from transformers import LlamaForCausalLM, LlamaTokenizer

tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
model = LlamaForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf", quantization_config=bnb_config, device_map = 'auto')

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

(…)-hf/resolve/main/special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

(…)at-hf/resolve/main/tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

(…)ma-2-7b-chat-hf/resolve/main/config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

(…)esolve/main/model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

(…)t-hf/resolve/main/generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [12]:
p = pipeline(
    "text-generation",
    tokenizer = tokenizer,
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [7]:
lit_context = "Biliary atresia (BA) is a progressive inflammatory process of the biliary tree resulting in biliary obstruction. No single known genetic or environmental factor has been established to cause BA. Cystic fibrosis (CF) is a rare cause of neonatal cholestasis, and it has never been described in familial BA cases. Here, we investigate two siblings of first-degree consanguineous parents presenting with neonatal BA. Shortly after the Kasai operation, the proband developed severe respiratory symptoms attributable to a missed CF diagnosis. This was discovered after re-investigating the family history, which revealed a first-degree cousin with CF who did not manifest BA. Afterwards, we identified a pathogenic variant (DeltaF508) in CFTR in both BA-affected siblings along with their cousin. This intrigued us to study the molecular etiology behind the familial BA presentations, which exclusively contributed to BA-pathogenesis in BA-CF-affected siblings and not in their CF-only affected cousin. We applied a multistep approach to investigate the variant profile of both siblings' and their cousin's exomes. We curated the genes whose variants were shared by the BA-CF siblings but absent or heterozygous in their CF-only-affected cousin. Consequently, we identified three candidate genes (SNAPC4, UCK1, and ZHX2) besides CFTR. We propose that these genes act cumulatively or individually in inducing BA-pathogenesis-either by aggravating the biliary damage in the context of CF or increasing the susceptibility of BA as a separate CF-comorbidity. To our knowledge, this is the first report of DeltaF508 in CFTR with familial neonatal BA cases. \n"

In [13]:

sequences = p(
    'From the following paragraph, identify any entiies. Do not add any information: \n{}'.format(lit_context),
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=2000,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")


Result: From the following paragraph, identify any entiies. Do not add any information: 
Biliary atresia (BA) is a progressive inflammatory process of the biliary tree resulting in biliary obstruction. No single known genetic or environmental factor has been established to cause BA. Cystic fibrosis (CF) is a rare cause of neonatal cholestasis, and it has never been described in familial BA cases. Here, we investigate two siblings of first-degree consanguineous parents presenting with neonatal BA. Shortly after the Kasai operation, the proband developed severe respiratory symptoms attributable to a missed CF diagnosis. This was discovered after re-investigating the family history, which revealed a first-degree cousin with CF who did not manifest BA. Afterwards, we identified a pathogenic variant (DeltaF508) in CFTR in both BA-affected siblings along with their cousin. This intrigued us to study the molecular etiology behind the familial BA presentations, which exclusively contributed to

In [14]:

sequences = p(
    'From the following paragraph, identify any entities and their relations. Do not add any information: \n{}'.format(lit_context),
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=2000,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: From the following paragraph, identify any entities and their relations. Do not add any information: 
Biliary atresia (BA) is a progressive inflammatory process of the biliary tree resulting in biliary obstruction. No single known genetic or environmental factor has been established to cause BA. Cystic fibrosis (CF) is a rare cause of neonatal cholestasis, and it has never been described in familial BA cases. Here, we investigate two siblings of first-degree consanguineous parents presenting with neonatal BA. Shortly after the Kasai operation, the proband developed severe respiratory symptoms attributable to a missed CF diagnosis. This was discovered after re-investigating the family history, which revealed a first-degree cousin with CF who did not manifest BA. Afterwards, we identified a pathogenic variant (DeltaF508) in CFTR in both BA-affected siblings along with their cousin. This intrigued us to study the molecular etiology behind the familial BA presentations, which exclu

In [17]:

sequences = p(
    'From the following paragraph, identify any biomedical entities and their relations: \n{}'.format(lit_context),
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=2000,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")


Result: From the following paragraph, identify any biomedical entities and their relations: 
Biliary atresia (BA) is a progressive inflammatory process of the biliary tree resulting in biliary obstruction. No single known genetic or environmental factor has been established to cause BA. Cystic fibrosis (CF) is a rare cause of neonatal cholestasis, and it has never been described in familial BA cases. Here, we investigate two siblings of first-degree consanguineous parents presenting with neonatal BA. Shortly after the Kasai operation, the proband developed severe respiratory symptoms attributable to a missed CF diagnosis. This was discovered after re-investigating the family history, which revealed a first-degree cousin with CF who did not manifest BA. Afterwards, we identified a pathogenic variant (DeltaF508) in CFTR in both BA-affected siblings along with their cousin. This intrigued us to study the molecular etiology behind the familial BA presentations, which exclusively contribute

In [33]:
sequences = p(
    """
    From the paragraph under CONTEXT, identify any biomedical entities and their relations and output them as knowledge graph in Resource Description Format (RDF) in JSON.  \nCONTEXT:{}
    """.format(lit_context),
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    # temperature = 0.1, ## we can play around with temperature too
    eos_token_id=tokenizer.eos_token_id,
    max_length=4096,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")


Result: 
    From the paragraph under CONTEXT, identify any biomedical entities and their relations and output them as knowledge graph in Resource Description Format (RDF) in JSON.  
CONTEXT:Biliary atresia (BA) is a progressive inflammatory process of the biliary tree resulting in biliary obstruction. No single known genetic or environmental factor has been established to cause BA. Cystic fibrosis (CF) is a rare cause of neonatal cholestasis, and it has never been described in familial BA cases. Here, we investigate two siblings of first-degree consanguineous parents presenting with neonatal BA. Shortly after the Kasai operation, the proband developed severe respiratory symptoms attributable to a missed CF diagnosis. This was discovered after re-investigating the family history, which revealed a first-degree cousin with CF who did not manifest BA. Afterwards, we identified a pathogenic variant (DeltaF508) in CFTR in both BA-affected siblings along with their cousin. This intrigued us 

In [34]:
sequences = p(
    """
    From the paragraph under CONTEXT, identify any gene-disease relations and output them as knowledge graph in Resource Description Format (RDF) in JSON.  \nCONTEXT:{}
    """.format(lit_context),
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    # temperature = 0.1, ## we can play around with temperature too
    eos_token_id=tokenizer.eos_token_id,
    max_length=4096,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")


Result: 
    From the paragraph under CONTEXT, identify any gene-disease relations and output them as knowledge graph in Resource Description Format (RDF) in JSON.  
CONTEXT:Biliary atresia (BA) is a progressive inflammatory process of the biliary tree resulting in biliary obstruction. No single known genetic or environmental factor has been established to cause BA. Cystic fibrosis (CF) is a rare cause of neonatal cholestasis, and it has never been described in familial BA cases. Here, we investigate two siblings of first-degree consanguineous parents presenting with neonatal BA. Shortly after the Kasai operation, the proband developed severe respiratory symptoms attributable to a missed CF diagnosis. This was discovered after re-investigating the family history, which revealed a first-degree cousin with CF who did not manifest BA. Afterwards, we identified a pathogenic variant (DeltaF508) in CFTR in both BA-affected siblings along with their cousin. This intrigued us to study the mole

In [35]:
sequences = p(
    """
    From the paragraph under CONTEXT, identify any gene-disease relations and output them as knowledge graph in Resource Description Format (RDF).  \nCONTEXT:{}
    """.format(lit_context),
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    # temperature = 0.1, ## we can play around with temperature too
    eos_token_id=tokenizer.eos_token_id,
    max_length=4096,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")


Result: 
    From the paragraph under CONTEXT, identify any gene-disease relations and output them as knowledge graph in Resource Description Format (RDF).  
CONTEXT:Biliary atresia (BA) is a progressive inflammatory process of the biliary tree resulting in biliary obstruction. No single known genetic or environmental factor has been established to cause BA. Cystic fibrosis (CF) is a rare cause of neonatal cholestasis, and it has never been described in familial BA cases. Here, we investigate two siblings of first-degree consanguineous parents presenting with neonatal BA. Shortly after the Kasai operation, the proband developed severe respiratory symptoms attributable to a missed CF diagnosis. This was discovered after re-investigating the family history, which revealed a first-degree cousin with CF who did not manifest BA. Afterwards, we identified a pathogenic variant (DeltaF508) in CFTR in both BA-affected siblings along with their cousin. This intrigued us to study the molecular et

In [39]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [42]:
sequences = p(
    """
    From the paragraph under CONTEXT, identify any gene-disease relations and output them as knowledge graph in Resource Description Format (RDF).  \nCONTEXT:{}
    """.format(lit_context),
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    # temperature = 0.1, ## we can play around with temperature too
    eos_token_id=tokenizer.eos_token_id,
    max_length=4096,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")


Result: 
    From the paragraph under CONTEXT, identify any gene-disease relations and output them as knowledge graph in Resource Description Format (RDF).  
CONTEXT:Biliary atresia (BA) is a progressive inflammatory process of the biliary tree resulting in biliary obstruction. No single known genetic or environmental factor has been established to cause BA. Cystic fibrosis (CF) is a rare cause of neonatal cholestasis, and it has never been described in familial BA cases. Here, we investigate two siblings of first-degree consanguineous parents presenting with neonatal BA. Shortly after the Kasai operation, the proband developed severe respiratory symptoms attributable to a missed CF diagnosis. This was discovered after re-investigating the family history, which revealed a first-degree cousin with CF who did not manifest BA. Afterwards, we identified a pathogenic variant (DeltaF508) in CFTR in both BA-affected siblings along with their cousin. This intrigued us to study the molecular et

In [ ]:
'''
|SNP| Diseases|
|rs113993960| BRONCHIECTASIS|
|rs113993960|Stenosis of duodenum|
|rs113993960| Congenital bilateral aplasia of vas deferens|
|rs113993960|Hereditary pancreatitis|
|rs113993960|Recurrent pancreatitis|
|rs113993960|Cystic Fibrosis|
|rs199473282|LONG QT SYNDROME 3|
|rs199473282|Brugada Syndrome (disorder)|
|rs199473282|Brugada Syndrome 1|
|rs199473282|Long QT Syndrome|
|rs199473282|Hereditary bundle branch system defect|
|rs121909211|Plaque, Amyloid|
|rs121909211|Familial Amyloid Polyneuropathy, Type V|
|rs121909211|Corneal dystrophy|
|rs121909211|Corneal deposit|
|rs121909211|Corneal Dystrophy, Lattice Type IIIA|
|rs121909211|Dystrophy, granular|
|rs121909211|Reis-Bucklers' corneal dystrophy|
|rs121909211|Corneal guttata|
|rs121909211|Lattice corneal dystrophy Type I|
|rs121909211|Amyloidosis|
|rs121909211|Neoplasms|
|rs121909211|Granular Dystrophy, Corneal|
|rs121909211|Thiel-Behnke corneal dystrophy|
|rs121909211|Stromal Dystrophies, Corneal|
|rs121909211|Avellino corneal dystrophy|
'''